In [1]:
#model parameters
import numpy as np
import samples as ls
import math
import model_types

#sgd 1_4, 2_4, 0_5, 10/11/12_34

#adam: 1/2_7, 0_6
#adam trained: 0_6, 2_7

#used networks:
# 0_6
# 1/2_?
# 10/11_34
# 12_36
# 11_? 
# 11_?

load_test = False
predict_mode = False

load_rnn = False if not predict_mode else False
load_only_weights = False if not load_rnn else False
model_num = 10
model_type = 102

train_network = True

#default model parameter
all_classes = False
accel,gyro,compass = 'xyz','xyz',''
overlap_step = 1
use_labels_data = True
use_lstm = True
use_sgd = False
use_rnn = True
use_labels_data = True
input_steps = None
unroll_steps = None
activation = 'relu'
load_samples = True
dropout_prob = 0
normalize = False
binary = False

model_id = '%s_%s' % (model_type, model_num)
model_path = 'tmp/keraslabel/nn-%s.hdf5' % model_id
weights_path = 'tmp/keraslabel/nn-%s_weights.hdf5' % model_id
labels,sample_ids,filename = model_types.get_labels(model_type, load_test)

if model_num == 0:
    hidden_layers = [50]
    unroll_steps = 100
    overlap_steps = 50
    all_classes = True
    load_samples = False
elif model_num == 1:
    hidden_layers = [6]
elif model_num == 2:
    unroll_steps = 100
    overlap_steps = 25
    hidden_layers = [50]
elif model_num == 4:
    unroll_steps = 10
    overlap_steps = 1
    hidden_layers = [5]
elif model_num == 10:
    hidden_layers = [6]
    binary = True
    
    
input_dim = len(accel)+len(gyro)+len(compass)

assert not binary or not all_classes

classes = len(labels) if all_classes else 1 if binary else 2

#percentage used for test data
test_rate = 0.1

learning_rate = 0.01

print('model: %s (type %s, num %s)' % (model_id,model_type,model_num))
print('%s classes, input_dim %s' % (classes, input_dim))

model: 102_10 (type 102, num 10)
1 classes, input_dim 6


In [2]:
#load samples
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)
    
    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)

    samples.convert_to_input(size = None, accel=accel, gyro=gyro, compass=compass)

samples.unroll(unroll_steps, overlap_step)
samples.convert_to_onehot()
samples.split_test(test_rate, random=False)
if not use_rnn: samples.flatten()
    
    
sumlen = 0
for key in labels:
    print('%s: %s / %s' % (key, len(samples.data[key]), len(samples.test[key])))
    if key != 'straight': sumlen += len(samples.data[key])
 
#extend straight
data = samples.data['straight'][:]
data2 = samples.labels_data['straight'][:]
while len(samples.data['straight']) < sumlen:
    samples.data['straight'].extend(data)
    samples.labels_data['straight'].extend(data2)
print(len(samples.data['straight']))

if not all_classes:
    for key in labels:
        for sample in range(len(samples.labels_data[key])):
            for s in range(len(samples.labels_data[key][sample])):
                for step in range(len(samples.labels_data[key][sample][s])):
                    if key == 'straight':
                        samples.labels_data[key][sample][s][step] = [1.0,0.0] if not binary else [0]
                    else:
                        samples.labels_data[key][sample][s][step] = [0.0,1.0] if not binary else [1]
        for sample in range(len(samples.labels_test[key])):
            for s in range(len(samples.labels_test[key][sample])):
                for step in range(len(samples.labels_test[key][sample][s])):
                    if key == 'straight':
                        samples.labels_test[key][sample][s][step] = [1.0,0.0] if not binary else [0]
                    else:
                        samples.labels_test[key][sample][s][step] = [0.0,1.0] if not binary else [1]
                        
                        
#prepare training data
xtrain,ytrain,seqlen = samples.get_all(padding=False, use_labels_data=use_labels_data)
xtest,ytest,seqlentest = samples.get_test(padding=False, use_labels_data=use_labels_data)


#normalize data
if normalize:
    
    mean = [0]*input_dim
    maxx = [-100000]*input_dim
    minx = [1000000]*input_dim
    for dataset in [xtrain,xtest]:
        for x in dataset:
            for d in input_dim:
                mean += x[d]
                if maxx[d] < x[d]: maxx[d] = x[d]
                if minx[d] > x[d]: minx[d] = x[d]

    for d in input_dim:
        mean[d] = mean[d]/(len(xtrain)+len(xtest))
        maxx[d] -= mean[d]
        minx[d] -= mean[d]
        
    
    def norm(x):
        for d in input_dim:
            x[d] -= mean[d]
            x[d] = (x[d] - minx[d]) / (maxx[d] - minx[d])
            x[d] = x[d]*2 - 1
        
    for dataset in [xtrain,xtest]:
        for x in dataset:
            norm(x)

    print(mean)
    print(maxx)
    print(minx)



if unroll_steps is None:
    from keras.preprocessing import sequence
    xtrain = sequence.pad_sequences(xtrain)
    xtest = sequence.pad_sequences(xtest)
    #not efficient but whatevs
    if len(xtrain[0]) > len(xtest[0]):
        input_steps = len(xtrain[0])
        xtest = sequence.pad_sequences(xtest, maxlen=input_steps)
    elif len(xtrain[0]) < len(xtest[0]):
        input_steps = len(xtest[0])
        xtrain = sequence.pad_sequences(xtrain, maxlen=input_steps)
    else:
        input_steps = len(xtrain[0])
    
    ytrain = sequence.pad_sequences(ytrain, maxlen=input_steps)
    ytest = sequence.pad_sequences(ytest, maxlen=input_steps)

straight: 36 / 4
curve_m_l: 54 / 6
curve_m_r: 54 / 6
wall: 90 / 10
foot: 72 / 8
object_l: 72 / 8


Using TensorFlow backend.


object_r: 72 / 8
432


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Masking
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

if load_rnn and not load_only_weights:
    model = load_model(model_path)
    model.load_weights(weights_path)
else:
    model = Sequential()
    last_act = 'sigmoid' if binary else 'softmax'
    if unroll_steps is None:
        model.add(Masking(input_shape=(input_steps, input_dim)))
        
        if use_lstm:
            model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       return_sequences=use_labels_data))
        else:
            model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       return_sequences=use_labels_data))
                     
        if use_labels_data:
            for k in hidden_layers:
                model.add(TimeDistributed(Dense(k, activation=activation)))
            model.add(TimeDistributed(Dense(classes, activation=last_act)))
        else:
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation=last_act))
        
    elif use_rnn:
        if use_lstm:
            if predict_mode:
                model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       batch_input_shape=(1, input_steps, input_dim), return_sequences=use_labels_data, stateful=True))
            else:
                model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
        else:
            if predict_mode:
                model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       batch_input_shape=(1, input_steps, input_dim), return_sequences=use_labels_data, stateful=True))
            else:
                model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
                     
        if use_labels_data:
            for k in hidden_layers:
                model.add(TimeDistributed(Dense(k, activation=activation)))
            model.add(TimeDistributed(Dense(classes, activation=last_act)))
        else:
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation=last_act))
    else:
        if not hidden_layers:
            model.add(Dense(classes, activation=last_act, input_shape=(input_steps*input_dim,)))
        else:
            model.add(Dense(hidden_layers.pop(0), activation=activation, input_shape=(input_steps*input_dim,)))
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation=last_act))
   
        
    

    if use_sgd: opt = optimizers.SGD(lr=learning_rate)
    else: opt = 'adam'
        
    if binary:
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    if predict_mode or load_only_weights:
        model.load_weights(weights_path)
    else:
        model.save(model_path)
        model.save_weights(weights_path)
        


In [4]:


if not predict_mode:
    def train(epochs, batch_size):
        mcp = ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, save_weights_only=True)
        model.fit(xtrain, ytrain,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(xtest, ytest),
                  callbacks=[mcp])
        score, acc = model.evaluate(xtest, ytest, batch_size=len(xtest))
        print('Test score: %s' % score)
        print('Test accuracy: %s' % acc)
        #model.save(model_path)
        #model.save_weights(weights_path)
        
    
    def test_total(batch_size=None):
        if batch_size == None:
            acc1 = model.evaluate(xtrain, ytrain, batch_size=len(xtrain), verbose=0)[1]
            acc2 = model.evaluate(xtest, ytest, batch_size=len(xtest), verbose=0)[1]
        else:
            acc1 = 0
            num = len(xtrain)
            for i in range(0,num,batch_size):
                size = min(batch_size, len(xtrain)-i)
                xtrain2 = xtrain[i:i+size]
                ytrain2 = ytrain[i:i+size]
                acc1 += (float(size)/num) * model.evaluate(xtrain2, ytrain2, batch_size=size, verbose=0)[1]
                
            acc2 = 0
            num = len(xtest)
            for i in range(0,num,batch_size):
                size = min(batch_size, len(xtest)-i)
                xtrain2 = xtest[i:i+size]
                ytrain2 = ytest[i:i+size]
                acc2 += (float(size)/num) * model.evaluate(xtrain2, ytrain2, batch_size=size, verbose=0)[1]
        acc = acc1*(1.0-test_rate) + acc2*test_rate
        print('Train accuracy: %s' % acc1)
        print('Test accuracy: %s' % acc2)
        print('Total accuracy: %s' % acc)
        
    def test():
        score, acc = model.evaluate(xtest, ytest, batch_size=len(xtest))
        print('Test score: %s' % score)
        print('Test accuracy: %s' % acc)
        
    
    def per_label_acc(test=True):
        if test:
            x,y = xtest,ytest
        else:
            x,y = xtrain,ytrain
        accs = [0] * classes
        counts = [0] * classes
        
        print_step = len(y)/10
        for i in range(len(y)):
            acc = model.evaluate(np.array([x[i]]), np.array([y[i]]), batch_size=1, verbose=0)[1]
            c = 0
            if use_labels_data:
                for y_oh in y[i]:
                    c = np.argmax(y_oh)
                    accs[c] += acc
                    counts[c] += 1
            else:
                c = np.argmax(y[i])
                accs[c] += acc
                counts[c] += 1
            if i%print_step==0:
                print('Progress: %s' % (float(i)/len(y)))
        
        acc_total = 0
        for i in range(classes):
            acc = accs[i]/counts[i]
            print('%sx %s: %s' % (counts[i],labels[i],acc))
            acc_total += acc*1.0/classes
        print('Total test acc: %s' % acc_total)
    
def feed_seq(s, as_prob=False):
    res = model.predict(np.array([s]),batch_size=1)[0]
    resarg = []
    for r in res:
        if as_prob: resarg.append(r)
        else: resarg.append(np.argmax(r))
    return resarg


In [5]:
if predict_mode:
    if False:
        import time
        start = time.time()
        for i in range(10000):
            z = [0,0,0,0,0,0]
            v = [z]*1
            feed_seq(v)
        dt = time.time() - start
        print('%s hz' % (1/(dt/10000)))

    if True:
        for key in samples.test:
            model.reset_states()
            xdata = samples.test[key][0][0]
            ydata = samples.labels_test[key][0][0]

            res = feed_seq(xdata)

            res_predicted = res
            res_expected = []
            acc = 0
            for i in range(len(res)):
                res_expected.append(np.argmax(ydata[i]))
                if res_predicted[len(res_expected)-1] == res_expected[-1]:
                    acc += 1.0
            print('%s: %s' % (key, acc/len(res)))
            print(res_predicted)
            print(res_expected)

In [6]:
if train_network and not predict_mode:
    #train(1000,32)
    
    #model.optimizer.lr.assign(0.00000000000001)
    train(2000,40)
    
    #per_label_acc()

Train on 846 samples, validate on 50 samples
Epoch 1/2000
846/846 [==============================] - 8s - loss: 0.6862 - acc: 0.5598 - val_loss: 0.7005 - val_acc: 0.2732
Epoch 2/2000
846/846 [==============================] - 8s - loss: 0.6830 - acc: 0.5799 - val_loss: 0.6878 - val_acc: 0.3385
Epoch 3/2000
846/846 [==============================] - 9s - loss: 0.6796 - acc: 0.6191 - val_loss: 0.6730 - val_acc: 0.5001
Epoch 4/2000
846/846 [==============================] - 9s - loss: 0.6767 - acc: 0.5149 - val_loss: 0.6606 - val_acc: 0.7275
Epoch 5/2000
846/846 [==============================] - 9s - loss: 0.6723 - acc: 0.6480 - val_loss: 0.6696 - val_acc: 0.4835
Epoch 6/2000
846/846 [==============================] - 10s - loss: 0.6663 - acc: 0.6570 - val_loss: 0.6628 - val_acc: 0.5122
Epoch 7/2000
846/846 [==============================] - 9s - loss: 0.6577 - acc: 0.6912 - val_loss: 0.6526 - val_acc: 0.5489
Epoch 8/2000
846/846 [==============================] - 9s - loss: 0.6429 - acc

846/846 [==============================] - 9s - loss: 0.3622 - acc: 0.8155 - val_loss: 0.4441 - val_acc: 0.6691
Epoch 66/2000
846/846 [==============================] - 9s - loss: 0.3593 - acc: 0.8202 - val_loss: 0.4780 - val_acc: 0.6626
Epoch 67/2000
846/846 [==============================] - 9s - loss: 0.3577 - acc: 0.8195 - val_loss: 0.4710 - val_acc: 0.6598
Epoch 68/2000
846/846 [==============================] - 9s - loss: 0.3675 - acc: 0.8152 - val_loss: 0.4981 - val_acc: 0.6495
Epoch 69/2000
846/846 [==============================] - 13s - loss: 0.3625 - acc: 0.8163 - val_loss: 0.4804 - val_acc: 0.6560
Epoch 70/2000
846/846 [==============================] - 9s - loss: 0.3614 - acc: 0.8173 - val_loss: 0.4779 - val_acc: 0.6608
Epoch 71/2000
846/846 [==============================] - 9s - loss: 0.3585 - acc: 0.8197 - val_loss: 0.4193 - val_acc: 0.6834
Epoch 72/2000
846/846 [==============================] - 9s - loss: 0.3562 - acc: 0.8220 - val_loss: 0.4369 - val_acc: 0.6751
Epoch

KeyboardInterrupt: 

In [ ]:
if True and not predict_mode:
    test_total(1000)
    #test_total(len(xtest))
    #test()
    per_label_acc()

In [ ]:
if predict_mode and True:
    %matplotlib notebook
    import matplotlib
    import matplotlib.pyplot as plt
    DIM = input_dim
    fig = plt.figure()
    ax = fig.add_subplot(111)
    #ax = fig.add_axes([0.1,0.1,0.75,0.75])
    plt.ion()
    fig.show()
    fig.canvas.draw()

    def draw_data_cutter(key, expected):
        model.reset_states()
        xdata = samples.test[key][0][0]
        ydata = samples.labels_test[key][0][0]

        res = feed_seq(xdata)
        print(res)
        
        ax.clear()
        ax.set_title('%s' % (key))
        ax.set_xlabel('time')

        data2 = np.array(xdata)
        data = []
        for i in range(DIM): data.append([])
        for d in data2:
            for i in range(DIM):
                data[i].append(d[i])

        for i in range(DIM):
            if i<3 and DIM>3:
                label = 'accel %s' % ('xyz'[i])
            else:
                label = 'gyro %s' % ('xyz'[i-3])
            ax.plot(range(len(data[i])),data[i], label=label)
        
        key_expected = 0 if key=='straight' else 1
        if all_classes:
            key_expected = labels.index(key)
        start_i = 0
        current_type = 0
        for i in range(len(res)):
            if current_type != res[i]:
                if current_type == key_expected:
                    ax.axvspan(start_i, i, color='green', alpha=0.1)
                elif current_type != key_expected and current_type != 0:
                    ax.axvspan(start_i, i, color='red', alpha=0.1)
                current_type = res[i]
                start_i = i
        if current_type == key_expected:
            ax.axvspan(start_i, len(res), color='green', alpha=0.1)
        elif current_type != key_expected and current_type != 0:
            ax.axvspan(start_i, i, color='red', alpha=0.1)
        
        ax.legend(loc='lower left')
        fig.canvas.draw()
        

In [ ]:
if predict_mode and True:
    draw_data_cutter('straight',False)
    #for label in labels:
    #    draw_data_cutter(label,False)
    #    plt.savefig('data-%s-%s.png' % (label, 'predicted'),pad_inches=0,bbox_inches='tight')